## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random
from urllib.request import urlretrieve

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from tqdm import tqdm
import seaborn as sns
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.ERROR)

In [6]:
# graph setting
sns.set()

In [7]:
# gpu
torch.cuda.is_available()

True

In [8]:
# global parameter
DATA_DIR="../dlwpt-code/data/"

## convolution

In [9]:
data_path = '../chapter7/data'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [10]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [11]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1) # <1>
img,_=cifar2[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

## nn.module

In [12]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.fc1=nn.Linear(8*8*8,32)
        self.fc2=nn.Linear(32,2)
        
    def forward(self,x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [13]:
def training_loop(n_epochs,optimizer,model,loss_fn,train_loader,device):
    for epoch in range(1,n_epochs+1):
        loss_train=0.0
        for imgs, labels in train_loader:
            imgs=imgs.to(device=device)
            labels=labels.to(device=device)
            outputs=model(imgs)
            loss=loss_fn(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train+=loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}'.format(
                epoch, loss_train / len(train_loader))) 
            

        with torch.no_grad():
            correct=0
            total=0
            for imgs,labels in val_loader:
                imgs=imgs.to(device=device)
                labels=labels.to(device=device)
                outputs=model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum()) 
            print("val_accuracy",correct/total)
                

In [14]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

device="cuda"
model = Net().to(device)  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 1,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    device=device,
)

Epoch 1, Training loss 0.5980315310939862
val_accuracy 0.769


## 複雑なモデル

In [15]:
nn.Dropout2d(p=0.2)

Dropout2d(p=0.2, inplace=False)

In [16]:
nn.BatchNorm2d(num_features=2)

BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

### ResNet

In [17]:
class ResBlock(nn.Module):
    def __init__(self,n_chans):
        super().__init__()
        self.conv=nn.Conv2d(n_chans,n_chans,kernel_size=3,padding=1,bias=False)
        self.batch_norm=nn.BatchNorm2d(num_features=n_chans)
        
    def forward(self,x):
        out=self.conv(out)
        out=self.batch_norm(out)
        out=torch.relu(out)
        return out+x

In [18]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.4)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv2_dropout = nn.Dropout2d(p=0.4)
        self.fc1 = nn.Linear(8 * 8 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = self.conv2_dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out